In [4]:
import pandas as pd
import numpy as np
import pickle
import os
from random import sample
from visual_semantic_embedding import tools, evaluation

#### Читаем текст

In [5]:
words_text = {}
words_list = ['1000', '4000']
for num_word in words_list:
    tmp = pd.read_csv('data/words/raw/top{0}_words.txt'.format(num_word), header = -1)
    if tmp.shape[0]<tmp.shape[1]:
        tmp = tmp.T
    tmp = [str(x).replace(' ', '') for x in tmp[0]]
    print np.array(tmp).shape
    words_text[num_word] = {'array': np.array(tmp),
                            'ind2word': {ind:word for ind, word in enumerate(tmp)}}

(998,)
(4473,)


##### Читаем модели

In [38]:
model_list = ['coco', 'f8k', 'f30k']
model_weights = {}
for model_name in model_list:
    model_weights[model_name] = tools.load_model('pretrained_models/{0}.npz'.format(model_name))

pretrained_models/coco.npz
Loading dictionary...
Creating inverted dictionary...
Loading model options...
Loading model parameters...
Compiling sentence encoder...
Compiling image encoder...
Packing up...
pretrained_models/f8k.npz
Loading dictionary...
Creating inverted dictionary...
Loading model options...
Loading model parameters...
Compiling sentence encoder...
Compiling image encoder...
Packing up...
pretrained_models/f30k.npz
Loading dictionary...
Creating inverted dictionary...
Loading model options...
Loading model parameters...
Compiling sentence encoder...
Compiling image encoder...
Packing up...


#### Делаем словарь с векторами

In [7]:
pretrained_vectors = {}
for num_word in words_list:
    for model_name in model_list:
        name_pretrain = '{0}_{1}'.format(num_word, model_name)
        pretrained_vectors[name_pretrain] = \
        tools.encode_sentences(model_weights[model_name],
                                       words_text[num_word]['array'])
        print pretrained_vectors[name_pretrain].shape
        pd.DataFrame(pretrained_vectors[name_pretrain]).to_csv(
            'data/words/coco/{0}.csv'.format(name_pretrain))
        

(998, 1024)
(998, 1024)
(998, 1024)
(4473, 1024)
(4473, 1024)
(4473, 1024)


#### Берем тексты, делаем для них признаки
###### https://gitlab.com/dikonov/Universal-Dictionary-of-Concepts/blob/master/data/csv/dict-nl-eng.csv

In [9]:
words_text['df'] = {}
table_semantic = pd.read_csv('data/words/raw/dict-nl-eng.csv', sep = '\t', header = -1)[8:]
table_semantic['word'] = table_semantic[0].apply(lambda x: x[1:-1] if ('-' not in x)
                                                 else '')

In [10]:
def get_tag_wordnet(x, ind):
    try:
        return x.split('(')[1].split(')')[0].split(',')[0].split('>')[ind]
    except:
        return ''
    
table_semantic['definition'] = table_semantic[2].apply(lambda x: get_tag_wordnet(x, 1))
table_semantic['concept'] = table_semantic[2].apply(lambda x: get_tag_wordnet(x, 2))
table_semantic['smth'] = table_semantic[2].apply(lambda x: get_tag_wordnet(x, 0))

##### Получаем для них вектора

In [11]:
table_semantic['concept'] = table_semantic[['definition', 'concept']].apply(
    lambda x:  x[0] if x[1] == '' else x[1], axis = 1)
table_semantic['definition'] = table_semantic[['definition', 'concept']].apply(
    lambda x:  '' if x[1] == x[0] else x[0], axis = 1)

table_semantic['pos'] = table_semantic[5].apply(lambda x: x[1:-1])
table_semantic['title'] = table_semantic['word'].apply(lambda x: x.istitle())

cols_list  = ['word', 'pos', 'concept', 'definition', 'smth', 'title']
table_semantic = table_semantic[cols_list]
#table_semantic[(table_semantic['title']) & (table_semantic['smth']!='iof')]['concept']
#table_semantic[(table_semantic['title']) & (table_semantic['smth']=='iof')]['concept']
#table_semantic['word'].drop_duplicates()

### Мержим со словами из 1к и 4 к

In [35]:
words_1000 = pd.DataFrame(words_text['1000']['array'], columns = ['words_1000'])
words_1000['ind'] = words_1000.index
words_4000 = pd.DataFrame(words_text['4000']['array'], columns = ['words_4000'])
words_4000['ind'] = words_4000.index

table_semantic_1000 = table_semantic.merge(words_1000, left_on = 'word', right_on = 'words_1000')
table_semantic_4000 = table_semantic.merge(words_4000, left_on = 'word', right_on = 'words_4000')

In [36]:
table_semantic_1000.to_csv('data/words/words1000_semantics.csv')
table_semantic_4000.to_csv('data/words/words4000_semantics.csv')

#### Читаем картинки, строим эмбеддинги и извлекаем из них VGG19 фичи

### Шапка для картинок

In [ ]:
%pylab inline
# чуть покрасивше картинки:
#pd.set_option('display.mpl_style', 'default')
import warnings
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['axes.grid'] = True
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Ubuntu'
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('font', weight='bold')
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
# чтобы был русский шрифт
from matplotlib import rc
font = {'family': 'Verdana',
        'weight': 'normal'}
rc('font', **font)

